In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [2]:
# 如果是linux系统
trainPath = os.getcwd()+'/train.csv'
testPath = os.getcwd()+'/test.csv'
train_df = pd.read_csv(trainPath, index_col=0)
test_df = pd.read_csv(testPath, index_col=0)

d:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# 如果是win系统

train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

d:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#去除y值
y_train = train_df.pop('acc_now_delinq')

In [5]:
#合并训练、测试集
all_df = pd.concat((train_df, test_df), axis = 0)

In [6]:
data = all_df

In [7]:
all_df.columns.values

array(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'earliest_cr_line', 'mths_since_last_record',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m'

In [10]:
#sub_grade

sub_grade_mapping = {'A1':0,'A2':1,'A3':2,'A4':3,'A5':4,
                     'B1':5,'B2':6,'B3':7,'B4':8,'B5':9,
                     'C1':10,'C2':11,'C3':12,'C4':13,'C5':14,
                     'D1':15,'D2':16,'D3':17,'D4':18,'D5':19,
                     'E1':20,'E2':21,'E3':22,'E4':23,'E5':24,
                     'F1':25,'F2':26,'F3':27,'F4':28,'F5':29,
                     'G1':30,'G2':31,'G3':32,'G4':33,'G5':34}

data.sub_grade = data.sub_grade.map(sub_grade_mapping)



In [11]:
data.issue_d = pd.to_datetime(data['issue_d'])
data.earliest_cr_line = pd.to_datetime(data['earliest_cr_line'])

In [12]:
data['issue_d_year']=data.issue_d.dt.year
data['issue_d_month'] = data.issue_d.dt.month
data['earliest_cr_line_year']=data.earliest_cr_line.dt.year
data['earliest_cr_line_month'] = data.earliest_cr_line.dt.month

In [13]:
data.issue_d.describe()

count                  887379
unique                    103
top       2015-10-01 00:00:00
freq                    48631
first     2007-06-01 00:00:00
last      2015-12-01 00:00:00
Name: issue_d, dtype: object

## 要删除的

In [14]:
'''
for delete: grade,zip_code,inq_last_12m,total_cu_tl,open_acc_6m,open_il_12m,open_il_24m,open_il_6m,total_bal_il,
open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,policy_code,emp_title,emp_title,desc,title,zip_code,dti_joint,
annual_inc_joint,varification_status_joint,il_uti,mths_since_last_record,mths_since_last_major_derog,addr_state
'''
feature_delete = {'grade','zip_code','inq_last_12m','total_cu_tl','open_acc_6m','open_il_12m','open_il_24m',
                  'open_il_6m','total_bal_il','open_rv_24m','open_rv_12m','max_bal_bc','all_util','inq_fi',
                 'emp_title','policy_code','desc','title','dti_joint','annual_inc_joint','addr_state','mths_since_rcnt_il',
                 'verification_status_joint','il_util','mths_since_last_record','mths_since_last_major_derog','issue_d','earliest_cr_line'}
data = data.drop(feature_delete,axis=1)

## 二值化

In [15]:
#for binarilization：pymnt_plan,term, application_type, initial_list_status
feature_for_binary = {'term','pymnt_plan','application_type','initial_list_status'}
def binary(feature_for_binary,data):
    for items in feature_for_binary: 
        mapping = {label:idx for idx,label in enumerate(set(data[items]))}  
        data[items] = data[items].map(mapping) 
    return data

In [16]:
binary(feature_for_binary,data)
temp = data.copy()

In [17]:
#emp_length
emp_length_mapping = {'10+ years':10,'9 years':9,'8 years':8,'7 years':7,'6 years':6,
                       '5 years':5,'4 years':4,'3 years':3,'2 years':2,'1 year':1,'< 1 year':0}
data.emp_length = data.emp_length.map(emp_length_mapping) 

## 填充null值

In [18]:
'''
少量null值的处理，用均值替代：emp_length, revol_util, annual_inc, total_acc, earliest_cr_line, tot_cur_bal, tot_coll_amt
total_rev_hi_lim, revol_util, pub_rec, collections_12_mths_ex_med
'''
feature_fill_null = {'emp_length','revol_util','annual_inc','total_acc','earliest_cr_line_year','earliest_cr_line_month','tot_cur_bal',
                    'tot_coll_amt','total_rev_hi_lim','revol_util','pub_rec','collections_12_mths_ex_med'}
def fill_null(feature_fill_null,data):
    for items in feature_fill_null:
        try:
            time.strptime(data[items].mode()[0],"%b-%Y")
            data[items] = data[items].fillna(data[items].mode()[0])
            print (data[items].mode()[0])
            print ('yes')
        except:
            data[items] = data[items].fillna(data[items].mean())
            print (data[items].mean())
            print('no')
#         if time.strptime(feature_dropna.mode()[0],"%b-%Y"):
#             data[items] = data[items].fillna(feature_dropna.mode())
#             print feature_dropna.mode()
#         else:
#             data[items] = data[items].fillna(feature_dropna.mean())
    return data

In [19]:
fill_null(feature_fill_null,data)

139458.1893358596
no
0.19530737589451735
no
1997.7001814391174
no
6.008694991651228
no
75027.58776076682
no
225.70261031934052
no
32068.620045415588
no
55.06769323142081
no
0.014379521073358324
no
6.756109765030705
no
25.268026145264
no


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,...,collection_recovery_fee,collections_12_mths_ex_med,application_type,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month
member_id,,,,,,,,,,,,,,,,,,,,,
13599771,8875.0,8875.0,8875.0,0,18.25,321.97,17,8.000000,MORTGAGE,55000.0,...,224.7408,0.0,1,0.00000,144784.000000,16700.000000,2014,3,1985.0,2.0
1491956,12500.0,12500.0,12475.0,0,8.90,396.92,4,1.000000,MORTGAGE,43000.0,...,0.0000,0.0,1,225.70261,139458.189336,32068.620045,2012,5,1992.0,7.0
64687987,33600.0,33600.0,33600.0,1,18.55,863.31,21,9.000000,RENT,101300.0,...,0.0000,0.0,1,770.00000,106215.000000,41300.000000,2015,9,2001.0,5.0
8446850,17000.0,17000.0,17000.0,1,9.71,358.78,5,10.000000,MORTGAGE,47840.0,...,0.0000,0.0,1,102.00000,129416.000000,22800.000000,2013,8,1988.0,9.0
58471388,14000.0,14000.0,14000.0,1,16.99,347.87,17,5.000000,RENT,30000.0,...,0.0000,0.0,1,0.00000,15589.000000,10300.000000,2015,7,1999.0,3.0
5337525,24000.0,24000.0,24000.0,1,17.27,599.96,14,10.000000,OWN,70000.0,...,0.0000,0.0,1,0.00000,311770.000000,24600.000000,2013,4,1983.0,4.0
30165519,12000.0,12000.0,12000.0,1,16.29,293.67,16,1.000000,MORTGAGE,40000.0,...,0.0000,0.0,1,1380.00000,10992.000000,8900.000000,2014,10,2006.0,3.0
1909391,23325.0,23325.0,23325.0,1,20.49,624.35,20,4.000000,MORTGAGE,110000.0,...,724.1454,0.0,1,0.00000,193824.000000,14800.000000,2012,11,1998.0,11.0
43182363,15000.0,15000.0,15000.0,1,10.49,322.34,7,3.000000,MORTGAGE,100000.0,...,0.0000,0.0,1,0.00000,241273.000000,36000.000000,2015,1,1996.0,8.0


In [20]:
#fill null = 0: collections_12_mths_ex_med
data.collections_12_mths_ex_med = data.collections_12_mths_ex_med.fillna(0)

In [21]:
temp1 = data.copy()

## 离散多类型变量

In [22]:
#for get_dummies: home_ownership, verification_status, loan_status,purpose

feature_for_dummy = {'home_ownership','verification_status','loan_status','purpose'}
def multi_get_dummies(feature_for_dummy,data):
    for items in feature_for_dummy: 
        data = data.join(pd.get_dummies(data[items],prefix=items))
        data.pop(items)
    return data    

In [23]:
loan_status_mapping = {'Charged Off':'CF','Fully Paid':'FP','Current':'Current','In Grace Period':'IGP','Late (31-120 days)':'L31-120',
                       'Issued':'Issued','Does not meet the credit policy. Status:Charged Off':'NotMeetCF','Does not meet the credit policy. Status:Fully Paid':'NotMeetFP',
                       'Default':'Default','Late (16-30 days)':'L16-30'}
data.loan_status = data.loan_status.map(loan_status_mapping) 

In [24]:
data.loan_status.unique()

array(['CF', 'FP', 'Current', 'IGP', 'L31-120', 'Issued', 'NotMeetCF',
       'NotMeetFP', 'Default', 'L16-30'], dtype=object)

In [25]:
data = multi_get_dummies(feature_for_dummy,data)

In [26]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,sub_grade,emp_length,annual_inc,pymnt_plan,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
member_id,,,,,,,,,,,,,,,,,,,,,
13599771,8875.0,8875.0,8875.0,0,18.25,321.97,17,8.0,55000.0,0,...,0,0,0,0,0,1,0,0,0,0
1491956,12500.0,12500.0,12475.0,0,8.90,396.92,4,1.0,43000.0,0,...,0,0,0,0,0,1,0,0,0,0
64687987,33600.0,33600.0,33600.0,1,18.55,863.31,21,9.0,101300.0,0,...,0,0,0,0,0,0,0,0,0,1
8446850,17000.0,17000.0,17000.0,1,9.71,358.78,5,10.0,47840.0,0,...,0,0,0,0,0,1,0,0,0,0
58471388,14000.0,14000.0,14000.0,1,16.99,347.87,17,5.0,30000.0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
data.isnull().sum().sum()

0

In [29]:
data.shape

(887379, 67)

In [30]:
processed_train_df = data.loc[train_df.index]
processed_test_df = data.loc[test_df.index]

In [31]:
processed_train_df.shape

(709903, 67)

In [32]:
processed_test_df.shape

(177476, 67)

In [39]:
processed_train_df2 = processed_train_df.join(y_train)

In [41]:
processed_train_df2.shape

(709903, 68)

In [42]:
processed_train_df2.to_csv("processed_train.csv" , encoding = "utf-8")
processed_test_df.to_csv("processed_test.csv" , encoding = "utf-8")